In [1]:
# =========================
# 0) Setup & imports
# =========================
import sys, json, re, time, logging
from pathlib import Path
import pandas as pd

logging.basicConfig(level=logging.INFO)

def find_project_root():
    p = Path.cwd()
    for parent in [p, *p.parents]:
        if (parent / "src").exists() or (parent / "data").exists():
            return parent
    return Path.cwd()

project_root = find_project_root()
src_path = project_root / "src"
if src_path.exists() and str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

data_dir = project_root / "data"
out_dir = project_root / "outputs"
out_dir.mkdir(parents=True, exist_ok=True)

print("project_root:", project_root)
print("src_path    :", src_path if src_path.exists() else "(absent)")
print("data_dir    :", data_dir)
print("outputs     :", out_dir)


project_root: c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector
src_path    : c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\src
data_dir    : c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\data
outputs     : c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\outputs


In [2]:
# =========================
# 1) Import L2 ENHANCED (robuste)
# =========================
try:
    # Essayer la version enhanced avec classification intelligente
    from detection.level2_intelligent.level2_intelligent_processor import IntelligentProcessor as Level2SimplifiedProcessor
    print("NIVEAU 2 ENHANCED importé (classification intelligente)")
except ImportError:
    try:
        # Fallback vers version simplifiée
        from detection.level2_simplified.level2_simplified_processor import Level2SimplifiedProcessor
        print("Niveau 2 importé ")
    except Exception as e:
        raise ImportError(
            "Impossible d'importer Level2 depuis 'src/detection/...'. "
            f"Vérifie que 'src' est bien au bon endroit. Erreur: {e}"
        )

Niveau 2 importé 


In [3]:
# =========================
# 2) Chargement Level-1
# =========================
lvl1_csv_candidates = [
    data_dir / "detection" / "level1_heuristic_results.csv",
    Path("/mnt/data/level1_heuristic_results.csv"),
]
lvl1_stats_candidates = [
    data_dir / "detection" / "level1_heuristic_stats.json",
    Path("/mnt/data/level1_heuristic_stats.json"),
]

level1_usable_file = next((p for p in lvl1_csv_candidates if p.exists()), None)
level1_stats_file  = next((p for p in lvl1_stats_candidates if p.exists()), None)
assert level1_usable_file is not None, "level1_heuristic_results.csv introuvable"
assert level1_stats_file  is not None, "level1_heuristic_stats.json introuvable"

df_level1 = pd.read_csv(level1_usable_file)
with open(level1_stats_file, "r", encoding="utf-8") as f:
    level1_stats = json.load(f)

print(f"Données Level 1: {len(df_level1)} résumés | {len(df_level1.columns)} colonnes")
# Adapter pour la colonne is_suspect (logique inversée par rapport à production_ready)
prod_ready = int((df_level1["is_suspect"] == False).sum())
print(f"Production ready: {prod_ready}/{len(df_level1)} ({prod_ready/len(df_level1)*100:.1f}%)")

grade_dist = df_level1["original_grade"].value_counts().sort_index()
for g, c in grade_dist.items():
    # Calculer directement depuis les données level1 (car grade_correlation absent du stats.json)
    grade_suspects = len(df_level1[(df_level1["original_grade"] == g) & (df_level1["is_suspect"] == True)])
    pct = (grade_suspects/c*100) if c else 0
    print(f"  {g}: {c} résumés ({grade_suspects} suspects, {pct:.1f}%)")

Données Level 1: 372 résumés | 15 colonnes
Production ready: 175/372 (47.0%)
  A: 62 résumés (2 suspects, 3.2%)
  A+: 60 résumés (1 suspects, 1.7%)
  B: 11 résumés (7 suspects, 63.6%)
  B+: 158 résumés (122 suspects, 77.2%)
  C: 17 résumés (17 suspects, 100.0%)
  D: 64 résumés (48 suspects, 75.0%)


In [4]:
# =========================
# 3) Entrées L2 de base - CORRIGÉ
# =========================
all_summaries_simplified = []
for _, row in df_level1.iterrows():
    # Utiliser directement is_suspect comme indicateur de défaut (pas d'inversion)
    # Si is_suspect=True, alors production_ready=False
    # Si is_suspect=False, alors production_ready=True
    production_ready = not bool(row["is_suspect"])  # Conversion explicite pour clarté
    
    all_summaries_simplified.append({
        "id": row["id"],
        "original_grade": row["original_grade"],
        "coherence": row["coherence"],
        "factuality": row["factuality"],
        "num_issues": row["num_issues"],
        "production_ready": production_ready,
        # Ajouter les métrics level1 pour meilleure classification level2
        "confidence_score": row.get("confidence_score", 0.5),
        "risk_level": row.get("risk_level", "unknown"),
        "word_count": row.get("word_count", 0),
        # "metadata" et "source_id" ajoutés ensuite
    })

print(f"Entrées L2 construites : {len(all_summaries_simplified)}")
# Vérification cohérence avec level1
prod_ready_count = sum(1 for s in all_summaries_simplified if s["production_ready"])
print(f"Production ready L2: {prod_ready_count}/{len(all_summaries_simplified)} ({prod_ready_count/len(all_summaries_simplified)*100:.1f}%)")

Entrées L2 construites : 372
Production ready L2: 175/372 (47.0%)


In [5]:
# =========================
# 4) Outils ID & sémantique
# =========================
import re
from hashlib import sha1

def _norm(s: str) -> str:
    return re.sub(r"\s+", " ", (s or "").strip().lower())

def make_source_id(url: str=None, title: str=None, published: str=None, source: str=None) -> str | None:
    base = url or f"{_norm(source)}|{_norm(title)}|{_norm(published)}"
    return sha1(_norm(base).encode("utf-8")).hexdigest() if base else None

def tok(s: str) -> set:
    return set(re.findall(r"[a-zàâäéèêëîïôöùûüç0-9]+", _norm(s)))

def jaccard(a: set, b: set) -> float:
    if not a or not b: return 0.0
    inter = len(a & b); uni = len(a | b)
    return inter/uni if uni else 0.0


In [6]:
# =========================
# 5) Articles source - SIMPLIFIÉ
# =========================
# Cette section est simplifiée car nous n'avons pas les liens entre level1 et les articles source
# Le level2 fonctionnera avec les données level1 disponibles uniquement

print("[SIMPLIFIED] Section articles source désactivée")
print("[SIMPLIFIED] Level2 utilisera uniquement les données level1 disponibles")

# Variables vides pour compatibilité avec les cellules suivantes  
by_article_id = {}
text_by_source_id = {}
print("Articles indexés: 0 | source_ids: 0 | mode simplifié activé")

[SIMPLIFIED] Section articles source désactivée
[SIMPLIFIED] Level2 utilisera uniquement les données level1 disponibles
Articles indexés: 0 | source_ids: 0 | mode simplifié activé


In [7]:
# =========================
# 6A) Méta L1 -> source_id - SIMPLIFIÉ
# =========================
# Cette section est simplifiée car level1 ne contient pas les colonnes de métadonnées nécessaires

print("[SIMPLIFIED] Métadonnées level1 non disponibles")
print("[SIMPLIFIED] Aucune colonne source_url, source_title, published, source trouvée")
print("[L1 META] metadata ajoutée: 0 | source_id créés: 0 (mode simplifié)")

[SIMPLIFIED] Métadonnées level1 non disponibles
[SIMPLIFIED] Aucune colonne source_url, source_title, published, source trouvée
[L1 META] metadata ajoutée: 0 | source_id créés: 0 (mode simplifié)


In [8]:
print(sorted(df_level1.columns.tolist()))
df_level1.head(2)


['coherence', 'confidence_score', 'entities_detected', 'fact_check_candidates_count', 'factuality', 'id', 'is_suspect', 'level0_status', 'num_issues', 'original_grade', 'priority_score', 'processing_time_ms', 'risk_level', 'suspicious_entities', 'word_count']


,id,level0_status,original_grade,coherence,factuality,is_suspect,confidence_score,risk_level,processing_time_ms,word_count,entities_detected,suspicious_entities,fact_check_candidates_count,priority_score,num_issues
0,0_adaptive,validated,A,0.76,0.902675,False,0.75,low,89.374065,85,2,0,1,0.0,2
1,1_adaptive,validated,A,0.78,1.000000,False,0.90,low,50.805807,66,4,0,1,0.0,1


In [9]:
# =========================
# 8-17) Sections de mapping - DÉSACTIVÉES
# =========================
# Ces sections tentent de mapper level1_id vers summary_id puis article_id
# Elles sont désactivées car :
# 1. Pas de colonne 'text' dans level1
# 2. Pas de métadonnées source dans level1  
# 3. Décalage entre 372 level1_id et 186 summary_id de production

print("[SIMPLIFIED] Sections 8-17 désactivées (mapping complexe)")
print("[SIMPLIFIED] Le level2 fonctionnera sans source_id")
print("[SIMPLIFIED] Tous les résumés level1 seront traités directement")

# Variables pour compatibilité
filled_from_prod = 0
filled_from_bridge = 0
filled_meta_only = 0

print(f"[FINAL] items avec source_id: 0/{len(all_summaries_simplified)} (mode simplifié)")

[SIMPLIFIED] Sections 8-17 désactivées (mapping complexe)
[SIMPLIFIED] Le level2 fonctionnera sans source_id
[SIMPLIFIED] Tous les résumés level1 seront traités directement
[FINAL] items avec source_id: 0/372 (mode simplifié)


In [10]:
# Cellule désactivée - voir cellule 8

In [11]:
# Cellule désactivée - voir cellule 8

In [12]:
# Cellule désactivée - voir cellule 8

In [13]:
# Cellule désactivée - voir cellule 8

In [14]:
# Cellule désactivée - voir cellule 8

In [15]:
# Cellule désactivée - voir cellule 8

In [16]:
# =========================
# 6G) Bridge L1 id -> summary_id par le TEXTE - DÉSACTIVÉ 
# =========================
# PROBLÈME: df_level1 ne contient pas la colonne 'text' nécessaire
# Cette section est désactivée car level1_heuristic_results.csv ne contient pas le texte des résumés

print("[BRIDGE] Section désactivée - colonne 'text' manquante dans level1")
print("[BRIDGE] Aucun mapping L1_id -> summary_id possible sans le texte")

# Variables vides pour éviter les erreurs dans les cellules suivantes
l1id_to_sid = {}
filled_from_bridge = 0

print(f"[BRIDGE->PROD] source_id ajoutés via (L1->SID->AID): +{filled_from_bridge}")
print(f"[FINAL] items avec source_id: {sum(1 for s in all_summaries_simplified if s.get('source_id'))}/{len(all_summaries_simplified)}")

[BRIDGE] Section désactivée - colonne 'text' manquante dans level1
[BRIDGE] Aucun mapping L1_id -> summary_id possible sans le texte
[BRIDGE->PROD] source_id ajoutés via (L1->SID->AID): +0
[FINAL] items avec source_id: 0/372


In [17]:
# Cellule désactivée - voir cellule 8

In [18]:
# Cellule désactivée - voir cellule 8

In [19]:
# =========================
# 7) Exécuter Level-2
# =========================
import time

# Assure-toi que Level2SimplifiedProcessor est déjà importé (cf. début du notebook)
level2 = Level2SimplifiedProcessor(mode="balanced")

t0 = time.time()
valid_summaries, results = level2.process_batch(all_summaries_simplified)
dt = time.time() - t0

print(f"[L2] Terminé en {dt:.2f}s | Validés {len(valid_summaries)}/{len(all_summaries_simplified)}")
print("Répartition par tier:", level2.get_stats().get("tier_distribution"))


INFO:detection.level2_simplified.level2_simplified_processor:Seuils configurés pour mode balanced
INFO:detection.level2_simplified.level2_simplified_processor:Level 2 Simplifié initialisé en mode balanced
INFO:detection.level2_simplified.level2_simplified_processor:Traitement Level 2 Simplifié: 372 résumés
INFO:detection.level2_simplified.level2_simplified_processor:Level 2 Simplifié terminé: 159/372 validés (42.7%)


[L2] Terminé en 0.01s | Validés 159/372
Répartition par tier: {'EXCELLENT': 76, 'GOOD': 82, 'MODERATE': 133, 'CRITICAL': 81}


In [20]:
# =========================
# 8) DF résultats + merge source_id (+ priorité finale AMÉLIORÉE)
# =========================
import pandas as pd

# 8.1) Résultats L2 -> DataFrame
rows = []
for r in results:
    rows.append({
        "summary_id": r.summary_id,
        "tier": r.tier_classification.value,
        "is_valid": r.is_valid,
        "validation_confidence": r.validation_confidence,
        "grade_score": r.grade_score,
        "coherence_score": r.coherence_score,
        "factuality_score": r.factuality_score,
        "issues_count": r.issues_count,
        "processing_time_ms": r.processing_time_ms,
        "justification": r.level3_justification
    })
df_results_simplified = pd.DataFrame(rows)
print("df_results_simplified:", df_results_simplified.shape)

# 8.2) IDs provenant de nos entrées enrichies (après 6E/6G/backfills)
def _sid_of(x: dict):
    if not isinstance(x, dict):
        return None
    return x.get("id") or x.get("summary_id")

df_ids = (
    pd.DataFrame(
        [
            {"summary_id": _sid_of(s), "source_id": s.get("source_id"), 
             "original_grade": s.get("original_grade"), "production_ready": s.get("production_ready")}
            for s in all_summaries_simplified
            if _sid_of(s) is not None
        ]
    )
    .drop_duplicates("summary_id")
)

# 8.3) Fusion résultats + source_id  (→ crée df_results_with_ids)
df_results_with_ids = df_results_simplified.merge(df_ids, on="summary_id", how="left")
missing_after = int(df_results_with_ids["source_id"].isna().sum())
print(f"[MERGE] manquants source_id (après enrichissement): {missing_after}/{len(df_results_with_ids)}")

# =========================
# 8-bis) Recalibrage ROBUSTE de la priorité L3 (AMÉLIORÉ)
# =========================
def recompute_l3_priority_enhanced(row) -> float:
    """
    Règles améliorées basées sur les vraies métriques:
      - Base = 1 - validation_confidence
      - Boost par tier mais plus nuancé
      - Boost par grade original (D/C = +0.1, B/B+ = +0.05)
      - Malus si production_ready=True ET tier=EXCELLENT (-0.2)
      - Boost si production_ready=False (+0.1)
      - Si pas de source_id -> plancher à 0.70 (plus bas qu'avant)
      - Borné [0, 1]
    """
    conf = float(row.get("validation_confidence", 0) or 0)
    base = 1.0 - conf

    # Boost par tier (plus nuancé)
    tier = str(row.get("tier") or "").upper()
    if tier == "CRITICAL":
        base = max(base, 0.85)
    elif tier == "MODERATE":
        base = max(base, 0.60)
    elif tier == "GOOD":
        base = max(base, 0.40)  # Plus élevé qu'avant
    elif tier == "EXCELLENT":
        base = max(base, 0.25)  # Plus élevé qu'avant
    
    # Boost selon le grade original
    grade = str(row.get("original_grade", "")).upper()
    if grade in ["D", "C"]:
        base += 0.10  # Grades critiques
    elif grade in ["B", "B+"]:
        base += 0.05  # Grades moyens

    # Adjustment selon production_ready
    prod_ready = row.get("production_ready", True)
    if not prod_ready:
        base += 0.10  # Boost si pas production ready
    elif prod_ready and tier == "EXCELLENT":
        base -= 0.20  # Malus si excellent ET production ready (probable faux positif)

    # Si pas de source_id -> plancher plus bas
    no_sid = pd.isna(row.get("source_id")) or (str(row.get("source_id")) == "")
    if no_sid:
        base = max(base, 0.70)  # Plus bas qu'avant (0.80 -> 0.70)

    return float(min(1.0, max(0.0, round(base, 3))))

# Calcule la priorité finale améliorée
df_results_with_ids["level3_priority_final"] = df_results_with_ids.apply(recompute_l3_priority_enhanced, axis=1)

print("Priority (améliorée):")
print(df_results_with_ids["level3_priority_final"].describe())
print("Répartition par seuils:")
for seuil in [0.5, 0.7, 0.8, 0.85, 0.9]:
    count = (df_results_with_ids["level3_priority_final"] >= seuil).sum()
    print(f"  >= {seuil}: {count} cas ({count/len(df_results_with_ids)*100:.1f}%)")

df_results_simplified: (372, 10)
[MERGE] manquants source_id (après enrichissement): 372/372
Priority (améliorée):
count    372.000000
mean       0.781718
std        0.115288
min        0.700000
25%        0.700000
50%        0.750000
75%        0.750000
max        1.000000
Name: level3_priority_final, dtype: float64
Répartition par seuils:
  >= 0.5: 372 cas (100.0%)
  >= 0.7: 372 cas (100.0%)
  >= 0.8: 81 cas (21.8%)
  >= 0.85: 81 cas (21.8%)
  >= 0.9: 81 cas (21.8%)


In [21]:
df_results_with_ids

,summary_id,tier,is_valid,validation_confidence,grade_score,coherence_score,factuality_score,issues_count,processing_time_ms,justification,source_id,original_grade,production_ready,level3_priority_final
0,0_adaptive,GOOD,True,1.000000,0.9,0.760000,0.902675,2,0.0,Priorité standard basée sur confiance,None,A,True,0.70
1,1_adaptive,GOOD,True,1.000000,0.9,0.780000,1.000000,1,0.0,Priorité standard basée sur confiance,None,A,True,0.70
2,2_adaptive,EXCELLENT,True,1.000000,1.0,0.956000,1.000000,0,0.0,Priorité standard basée sur confiance,None,A+,True,0.70
3,3_adaptive,EXCELLENT,True,1.000000,0.9,0.876000,0.903475,1,0.0,Priorité standard basée sur confiance,None,A,True,0.70
4,4_adaptive,GOOD,True,1.000000,1.0,0.936000,1.000000,2,0.0,Priorité standard basée sur confiance,None,A+,True,0.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,181_confidence_weighted,MODERATE,False,0.941658,0.7,0.677476,0.938861,5,0.0,Grade B+ non production ready - traitement ML ...,None,B+,False,0.75
368,182_confidence_weighted,MODERATE,False,0.914382,0.7,0.649061,0.847942,5,0.0,Grade B+ non production ready - traitement ML ...,None,B+,False,0.75
369,183_confidence_weighted,CRITICAL,False,0.100000,0.2,0.301675,0.891555,6,0.0,Grade D non production ready - traitement ML c...,None,D,False,1.00
370,184_confidence_weighted,MODERATE,False,0.896209,0.7,0.653487,0.787365,5,0.0,Grade B+ non production ready - traitement ML ...,None,B+,False,0.75


In [22]:
df_results_with_ids["level3_priority_final"]

0      0.70
1      0.70
2      0.70
3      0.70
4      0.70
       ... 
367    0.75
368    0.75
369    1.00
370    0.75
371    0.75
Name: level3_priority_final, Length: 372, dtype: float64

In [23]:
len(df_results_with_ids[df_results_with_ids["level3_priority_final"] > 0.7])

210

In [24]:
# =========================
# 9) Cas prioritaires + compteurs (PRIORITÉ FINALE)
# =========================
PRIORITY_COL = "level3_priority_final"  # on utilise la priorité recalculée

priority_with_ids = df_results_with_ids[df_results_with_ids[PRIORITY_COL] > 0.7].copy()

missing_prio  = int(priority_with_ids["source_id"].isna().sum())
missing_total = int(df_results_with_ids["source_id"].isna().sum())

print(f"Manquants source_id (prioritaires): {missing_prio}/{len(priority_with_ids)}")
print(f"Manquants source_id (tous)        : {missing_total}/{len(df_results_with_ids)}")
print("Cutoffs — nb>0.7:", (df_results_with_ids[PRIORITY_COL] > 0.7).sum(),
      "| nb>0.85:", (df_results_with_ids[PRIORITY_COL] > 0.85).sum())


Manquants source_id (prioritaires): 210/210
Manquants source_id (tous)        : 372/372
Cutoffs — nb>0.7: 210 | nb>0.85: 81


In [25]:
# =========================
# 10) Exports (CSV/JSON + mapping L3)
# =========================
from pathlib import Path
import json

# chemins de sortie (out_dir doit être défini plus haut)
csv_all  = out_dir / "level2_simplified_results_with_ids.csv"
csv_prio = out_dir / "level2_simplified_priority_cases_with_ids.csv"
json_all = out_dir / "level2_output_with_source_id.json"
map_csv  = out_dir / "mapping_backfill_level2.csv"
map_l1   = out_dir / "mapping_level1id_to_source_id.csv"

# 10.1) résultats complets (avec la colonne level3_priority_final)
df_results_with_ids.to_csv(csv_all, index=False, encoding="utf-8")

# 10.2) sous-ensemble prioritaire (>0.7 sur la priorité finale)
(df_results_with_ids[df_results_with_ids["level3_priority_final"] > 0.7]
 .to_csv(csv_prio, index=False, encoding="utf-8"))

# 10.3) JSON pour L3 (inclut la colonne de priorité utilisée)
payload = {
    "summaries": all_summaries_simplified,                    # chaque item peut contenir 'metadata' + 'source_id'
    "results": df_results_with_ids.to_dict(orient="records"), # résultats L2 fusionnés + priorité finale
    "stats": level2.get_stats(),
    "priority_column": "level3_priority_final",
}
json_all.write_text(json.dumps(payload, ensure_ascii=False, indent=2), encoding="utf-8")

# 10.4) Mapping summary_id -> source_id (accélère L3)
(df_results_with_ids[["summary_id", "source_id"]]
 .drop_duplicates("summary_id")
 .to_csv(map_csv, index=False, encoding="utf-8"))

# 10.5) (bonus) Mapping level1_id -> source_id pour rejouer L3 directement depuis L1
l1_rows = [{"level1_id": s["id"], "source_id": s.get("source_id")} for s in all_summaries_simplified]
pd.DataFrame(l1_rows).to_csv(map_l1, index=False, encoding="utf-8")

print("ÉCRIT :")
print(" -", csv_all)
print(" -", csv_prio)
print(" -", json_all)
print(" -", map_csv)
print(" -", map_l1)


ÉCRIT :
 - c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\outputs\level2_simplified_results_with_ids.csv
 - c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\outputs\level2_simplified_priority_cases_with_ids.csv
 - c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\outputs\level2_output_with_source_id.json
 - c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\outputs\mapping_backfill_level2.csv
 - c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\outputs\mapping_level1id_to_source_id.csv


In [26]:
# =========================
# 11) Garde-fous finaux + Validation cohérence
# =========================
assert "summary_id" in df_results_with_ids.columns, "summary_id manquant dans df_results_with_ids"
assert df_results_with_ids["summary_id"].isna().sum() == 0, "Il y a des summary_id NaN (anormal)"
assert len(df_results_with_ids) == len(df_results_simplified), "Perte de lignes après merge (anormal)"
assert "level3_priority_final" in df_results_with_ids.columns, "colonne level3_priority_final absente"

coverage_all  = 100 * (1 - df_results_with_ids["source_id"].isna().mean())
coverage_prio = 100 * (1 - (priority_with_ids["source_id"].isna().mean() if len(priority_with_ids) else 0))

# Validation cohérence avec level1
l1_prod_ready = (df_level1["is_suspect"] == False).sum()
l2_valid = df_results_with_ids["is_valid"].sum()
l2_prod_ready = df_results_with_ids["production_ready"].sum()

print("\n-- KPI FINAUX --")
print(f"Total rows: {len(df_results_with_ids)}")
print(f"Couverture source_id (tous): {coverage_all:.1f}%")
print(f"Couverture source_id (prioritaires): {coverage_prio:.1f}%")

print("\n-- COHÉRENCE LEVEL1/LEVEL2 --")
print(f"Level1 production ready: {l1_prod_ready}/{len(df_level1)} ({l1_prod_ready/len(df_level1)*100:.1f}%)")
print(f"Level2 production ready: {l2_prod_ready}/{len(df_results_with_ids)} ({l2_prod_ready/len(df_results_with_ids)*100:.1f}%)")
print(f"Level2 validés: {l2_valid}/{len(df_results_with_ids)} ({l2_valid/len(df_results_with_ids)*100:.1f}%)")

# Analyse par grade des différences L1/L2
print("\n-- ANALYSE PAR GRADE (L1 production_ready vs L2 validés) --")
for grade in sorted(df_level1["original_grade"].unique()):
    l1_grade = df_level1[df_level1["original_grade"] == grade]
    l1_prod_count = (l1_grade["is_suspect"] == False).sum()
    l1_total = len(l1_grade)
    
    l2_grade = df_results_with_ids[df_results_with_ids["original_grade"] == grade]
    l2_valid_count = l2_grade["is_valid"].sum()
    l2_total = len(l2_grade)
    
    print(f"  {grade}: L1={l1_prod_count}/{l1_total} ({l1_prod_count/l1_total*100:.0f}%) | L2={l2_valid_count}/{l2_total} ({l2_valid_count/l2_total*100:.0f}%)")

print("\n-- RÉPARTITION TIERS --")
tier_counts = df_results_with_ids["tier"].value_counts().sort_values(ascending=False)
for tier, count in tier_counts.items():
    pct = count/len(df_results_with_ids)*100
    valid_in_tier = df_results_with_ids[df_results_with_ids["tier"] == tier]["is_valid"].sum()
    print(f"  {tier}: {count} cas ({pct:.1f}%) - {valid_in_tier} validés ({valid_in_tier/count*100:.0f}%)")


-- KPI FINAUX --
Total rows: 372
Couverture source_id (tous): 0.0%
Couverture source_id (prioritaires): 0.0%

-- COHÉRENCE LEVEL1/LEVEL2 --
Level1 production ready: 175/372 (47.0%)
Level2 production ready: 175/372 (47.0%)
Level2 validés: 159/372 (42.7%)

-- ANALYSE PAR GRADE (L1 production_ready vs L2 validés) --
  A: L1=60/62 (97%) | L2=60/62 (97%)
  A+: L1=59/60 (98%) | L2=59/60 (98%)
  B: L1=4/11 (36%) | L2=4/11 (36%)
  B+: L1=36/158 (23%) | L2=36/158 (23%)
  C: L1=0/17 (0%) | L2=0/17 (0%)
  D: L1=16/64 (25%) | L2=0/64 (0%)

-- RÉPARTITION TIERS --
  MODERATE: 133 cas (35.8%) - 4 validés (3%)
  GOOD: 82 cas (22.0%) - 79 validés (96%)
  CRITICAL: 81 cas (21.8%) - 0 validés (0%)
  EXCELLENT: 76 cas (20.4%) - 76 validés (100%)
